# Web Scraping

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import time
import random
import os

## Obtaining Last 20 years of Draft Results

In [ ]:
# List of last eligible 20 years to scrape data. 2020-2023 left out since their four-year
# rookie deal length are not yet completed.
YEARS = list(range(2000, 2020))

In [ ]:
dfs=[]
for yr in YEARS:
    try:
        url = f"https://www.pro-football-reference.com/years/{yr}/draft.htm"
        response = requests.get(url)
        stats_page = BeautifulSoup(response.text, "html.parser")

        headers = [th.getText() for th in stats_page.findAll('tr')[1].findAll('th')]
        headers = headers[1:]

        rows = stats_page.findAll('tr', class_=lambda table_rows: table_rows != 'thread')
        player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
        player_stats = player_stats[2:]

        df = pd.DataFrame(player_stats, columns=headers)

        dfs.append(df)

        print("Successful year scraped: " + str(yr))

        # Create first dataframe and then add subsequent dataframes
        if len(dfs)==1:
            comb_df = df
        else:
            comb_df = pd.concat([comb_df,df])

    except:
        # Notification of years that couldn't be read in
        print("error year:")
        print(yr)

Successful year scraped: 2000
Successful year scraped: 2001
Successful year scraped: 2002
Successful year scraped: 2003
Successful year scraped: 2004
Successful year scraped: 2005
Successful year scraped: 2006
Successful year scraped: 2007
Successful year scraped: 2008
Successful year scraped: 2009
Successful year scraped: 2010
Successful year scraped: 2011
Successful year scraped: 2012
Successful year scraped: 2013
Successful year scraped: 2014
Successful year scraped: 2015
Successful year scraped: 2016
Successful year scraped: 2017
Successful year scraped: 2018
Successful year scraped: 2019


In [ ]:
print(comb_df)

     Pick    Tm            Player   Pos   Age    To   AP1    PB    St   wAV  \
0       1   CLE    Courtney Brown    DE    22  2005     0     0     4    27   
1       2   WAS   LaVar Arrington    LB    22  2006     0     3     5    46   
2       3   WAS     Chris Samuels     T    23  2009     0     6     9    63   
3       4   CIN     Peter Warrick    WR    23  2005     0     0     4    27   
4       5   BAL       Jamal Lewis    RB    21  2009     1     1     9    69   
..    ...   ...               ...   ...   ...   ...   ...   ...   ...   ...   
258   253   WAS  Jordan Brailford    DE    23  2020     0     0     0     1   
259   254   ARI      Caleb Wilson    TE    23  2020     0     0     0     0   
260  None  None              None  None  None  None  None  None  None  None   
261  None  None              None  None  None  None  None  None  None  None   
262     1   ARI    Jalen Thompson    DB    21  2022     0     0     4    15   

     ...    Yds    TD   Rec   Yds    TD  Solo   Int

In [ ]:
# Create boolean masks that select rows where the "Pos" column is "WR" or "RB", respectively
wr_mask = comb_df["Pos"] == "WR"
rb_mask = comb_df["Pos"] == "RB"

# Create new dataframes that include only the selected rows
wr_df = comb_df[wr_mask]
rb_df = comb_df[rb_mask]

# Print the new dataframes to the console
print(wr_df)
print(rb_df)

    Pick   Tm            Player Pos Age    To AP1 PB  St wAV  ...  Yds TD  \
3      4  CIN     Peter Warrick  WR  23  2005   0  0   4  27  ...  360  2   
7      8  PIT   Plaxico Burress  WR  23  2012   0  0  10  70  ...   -7  0   
9     10  BAL     Travis Taylor  WR  22  2007   0  0   7  30  ...  232  0   
20    21  KAN  Sylvester Morris  WR  22  2000   0  0   1   6  ...    0  0   
28    29  JAX     R. Jay Soward  WR  22  2000   0  0   0   2  ...   28  0   
..   ...  ...               ...  ..  ..   ...  .. ..  ..  ..  ...  ... ..   
212  208  TAM      Scott Miller  WR  22  2022   0  0   1   9  ...   81  1   
241  236  SEA        John Ursua  WR  25  2019   0  0   0   0  ...    0  0   
242  237  CAR      Terry Godwin  WR  22  2020   0  0   0   0  ...    3  0   
244  239  MIN   Dillon Mitchell  WR  22         0  0   0      ...           
252  247  MIN   Olabisi Johnson  WR  22  2020   0  0   0   5  ...    6  0   

     Rec   Yds  TD Solo Int Sk   College/Univ                 
3    275  29

In [ ]:

# os.makedirs('Data-Collection/wr', exist_ok=True)  
# wr_df.to_csv('Data-Collection/wr/wr_draft_output.csv')

os.makedirs('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr', exist_ok=True)  
wr_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_draft_output.csv') 


os.makedirs('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb', exist_ok=True)  
rb_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb/rb_draft_output.csv')  


## Obtain Combine Results from last 20 years

In [ ]:
## Obtain combine results using Pandas' web scraping tool (uses Beautiful Soup inside of function).

## Use random time values to prevent too many HTTP requests
# time.sleep(random.choice([x/10 for x in range(3,8)]))

# URL of the NFL combine results page
combine_url = f"https://www.pro-football-reference.com/draft/{2020}-combine.htm"

combine2020_df = pd.read_html(combine_url)
combine2020_df[0].head()



,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)
0,Trey Adams,OL,Washington,College Stats,6-8,318,5.60,24.5,NaN,92,NaN,NaN,NaN
1,Hakeem Adeniji,OL,Kansas,College Stats,6-4,302,5.17,34.0,26,115,NaN,NaN,Cincinnati Bengals / 6th / 180th pick / 2020
2,McTelvin Agim,DL,Arkansas,College Stats,6-3,309,4.98,NaN,27,NaN,NaN,NaN,Denver Broncos / 3rd / 95th pick / 2020
3,Salvon Ahmed,RB,Washington,College Stats,5-11,197,4.62,34.5,NaN,120,NaN,NaN,NaN
4,Brandon Aiyuk,WR,Arizona State,College Stats,6-0,205,4.50,40.0,11,128,NaN,NaN,San Francisco 49ers / 1st / 25th pick / 2020


In [ ]:
## Obtain combine results for each of the players

combine_dfs=[]
for yr in YEARS:
    try:
        # URL of the NFL combine results page
        combine_url = f"https://www.pro-football-reference.com/draft/{yr}-combine.htm"
        combine_df_list = pd.read_html(combine_url)
        combine_df = combine_df_list[0]

        # Append to list for future use
        combine_dfs.append(combine_df)

        print("Successful year scraped: " + str(yr) + "  list length: " + str(len(combine_dfs)))

        # Create first pandas dataframe and then add subsequent dataframes
        if len(combine_dfs)==1:
            combo_df = combine_df
        else:
            combo_df = pd.concat([combo_df,combine_df])
    except:
        # Notification of years that couldn't be read in
        print("error year:")
        print(yr)

print(len(combine_dfs))



Successful year scraped: 2000  list length: 1
Successful year scraped: 2001  list length: 2
Successful year scraped: 2002  list length: 3
Successful year scraped: 2003  list length: 4
Successful year scraped: 2004  list length: 5
Successful year scraped: 2005  list length: 6
Successful year scraped: 2006  list length: 7
Successful year scraped: 2007  list length: 8
Successful year scraped: 2008  list length: 9
Successful year scraped: 2009  list length: 10
Successful year scraped: 2010  list length: 11
Successful year scraped: 2011  list length: 12
Successful year scraped: 2012  list length: 13
Successful year scraped: 2013  list length: 14
Successful year scraped: 2014  list length: 15
Successful year scraped: 2015  list length: 16
Successful year scraped: 2016  list length: 17
Successful year scraped: 2017  list length: 18
Successful year scraped: 2018  list length: 19
Successful year scraped: 2019  list length: 20
20


In [ ]:
wr_mask = combo_df["Pos"] == "WR"
rb_mask = combo_df["Pos"] == "RB"

# Create new dataframes that include only the selected rows
wr_combine_df = combo_df[wr_mask]
rb_combine_df = combo_df[rb_mask]

In [ ]:
wr_combine_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_combine_output.csv') 
rb_combine_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb/rb_combine_output.csv') 

## College Results

In case, this notebook needs to be rerun, and previously scraped data is now affected by overloading request limits, reload the wr and rb dataframes saved to the drive.

In [ ]:
wr_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_draft_output.csv')
rb_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb/rb_draft_output.csv')

### College WR Production

In [ ]:
# Practice with Marquise Brown
player = "Marquise Brown"

# Create a URL for the player's college stats page
url = "https://www.sports-reference.com/cfb/players/{}-1.html".format(player.replace(" ", "-").lower())

# Request the HTML content from the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find the table element containing the player's college stats. Rb's table is called rushing
table = soup.find("table", {"id": "receiving"})

# Extract the table headers
headers = [th.get_text() for th in table.find("thead").find_all("th")]
headers = headers[6:]
# Create a list to hold the college stats data
college_stats = []

# Extract the college stats row by row
for tr in table.find("tbody").find_all("tr"):
    row = [td.get_text() for td in tr.find_all("td")]
    college_stats.append(row)

# Convert the college stats to a pandas DataFrame
college_df = pd.DataFrame(college_stats, columns=headers)

# Add a "Player" column to the college stats DataFrame
college_df["Player"] = player

college_df.head()

,School,Conf,Class,Pos,G,Rec,Yds,Avg,TD,Att,Yds,Avg,TD,Plays,Yds,Avg,TD,Player
0,Oklahoma,Big 12,SO,WR,13,57,1095,19.2,7,1,0,0.0,0,58,1095,18.9,7,Marquise Brown
1,Oklahoma,Big 12,JR,WR,12,75,1318,17.6,10,2,0,0.0,0,77,1318,17.1,10,Marquise Brown


In [ ]:
## For each player obtain their college stats
# Create a list to hold the college stats dataframes
college_wr_dfs = []
error_wr_players = []

In [ ]:
from time import sleep
from tqdm import tqdm_notebook as tqdm



# Loop through each player in the draft results dataframe
for i, player in enumerate(tqdm(wr_df["Player"], desc="Scraping players")):   
    
        print(i)
        try:
            # Create a URL for the player's college stats page
            url = "https://www.sports-reference.com/cfb/players/{}-1.html".format(player.replace(" ", "-").lower())

            # Request the HTML content from the URL
            response = requests.get(url)

            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, "html.parser")

            # Find the table element containing the player's college stats. Rb's table is called rushing
            table = soup.find("table", {"id": "receiving"})

            # If the player did not have any college stats, skip to the next player
            if table is None:
                error_wr_players.append(player)
                continue

            # Extract the table headers
            headers = [th.get_text() for th in table.find("thead").find_all("th")]
            headers = headers[6:]

            # Create a list to hold the college stats data
            college_stats = []

            # Extract the college stats row by row
            for tr in table.find("tbody").find_all("tr"):
                row = [td.get_text() for td in tr.find_all("td")]
                college_stats.append(row)

            # Convert the college stats to a pandas DataFrame
            college_df = pd.DataFrame(college_stats, columns=headers)

            # Add a "Player" column to the college stats DataFrame
            college_df["Player"] = player

            # Add the college stats DataFrame to the list of college stats dataframes
            college_wr_dfs.append(college_df)

            # Create first pandas dataframe and then add subsequent dataframes
            if len(college_wr_dfs)==1:
                college_wr_df = college_df
            else:
                college_wr_df = pd.concat([college_wr_df,college_df])

            # Wait 1 minute after every 10 players to avoid hitting request limits
            if (i+1) % 10 == 0:
                print("Waiting for 1 minute...")
                sleep(60)            

        except:
            # Notification of years that couldn't be read in
            error_wr_players.append(player)
            # print("error player:")
            # print(player)

# Print the college stats dataframe to the console
print(college_wr_df)
print(error_wr_players)

college_wr_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_college_output_part4_4.csv') 




Scraping players:   0%|          | 0/181 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


NameError: ignored

In [ ]:
college_wr_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_college_total.csv') 


### RB College Production

In [ ]:
# Practice with Tevin Coleman
player = "Tevin Coleman"

# Create a URL for the player's college stats page
url = "https://www.sports-reference.com/cfb/players/{}-1.html".format(player.replace(" ", "-").lower())


# Request the HTML content from the URL
response = requests.get(url)
print(response.content[:500])

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find the table element containing the player's college stats. Rb's table is called rushing
table = soup.find("table", {"id": "rushing"})


# Extract the table headers
headers = [th.get_text() for th in table.find("thead").find_all("th")]
headers = headers[6:]
# Create a list to hold the college stats data
college_stats = []

# Extract the college stats row by row
for tr in table.find("tbody").find_all("tr"):
    row = [td.get_text() for td in tr.find_all("td")]
    college_stats.append(row)

# Convert the college stats to a pandas DataFrame
college_df = pd.DataFrame(college_stats, columns=headers)




b'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="" lang="en" class="no-js" >\n<head id="suppress_all_ads">\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n\n    <title>Rate Limited Request (429 error) | Sports-Reference.com</title>\n\n    <link rel="canonical" href="https://www.sports-reference.com/429.html" />\n\n<!-- CSS start -->\n <style>body,html{color:#000;f'


AttributeError: ignored

In [ ]:
## For each player obtain their college stats
# Create a list to hold the college stats dataframes
college_rb_dfs = []
error_rb_players = []

from time import sleep
from tqdm import tqdm_notebook as tqdm

# Loop through each player in the draft results dataframe
for i, player in enumerate(tqdm(rb_df["Player"], desc="Scraping players")):
    try:
        # Create a URL for the player's college stats page
        url = "https://www.sports-reference.com/cfb/players/{}-1.html".format(player.replace(" ", "-").lower())

        # Request the HTML content from the URL
        response = requests.get(url)

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Find the table element containing the player's college stats. Rb's table is called rushing
        table = soup.find("table", {"id": "rushing"})

        # If the player did not have any college stats, skip to the next player
        if table is None:
            error_rb_players.append(player)
            continue

        # Extract the table headers
        headers = [th.get_text() for th in table.find("thead").find_all("th")]
        headers = headers[6:]

        # Create a list to hold the college stats data
        college_stats = []

        # Extract the college stats row by row
        for tr in table.find("tbody").find_all("tr"):
            row = [td.get_text() for td in tr.find_all("td")]
            college_stats.append(row)

        # Convert the college stats to a pandas DataFrame
        college_df = pd.DataFrame(college_stats, columns=headers)

        # Add a "Player" column to the college stats DataFrame
        college_df["Player"] = player

        # Add the college stats DataFrame to the list of college stats dataframes
        college_rb_dfs.append(college_df)

        # Create first pandas dataframe and then add subsequent dataframes
        if len(college_rb_dfs)==1:
            college_rb_df = college_df
        else:
            college_rb_df = pd.concat([college_rb_df,college_df])

        # Wait 1 minute after every 10 players to avoid hitting request limits
        if (i+1) % 10 == 0:
            print("Waiting for 1 minute...")
            sleep(60)

    except:
        # Notification of years that couldn't be read in
        error_rb_players.append(player)
        # print("error player:")
        # print(player)

# Print the college stats dataframe to the console
print(college_rb_df)
print(error_rb_players)

<ipython-input-50-882a1c87a5d1>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, player in enumerate(tqdm(error_rb_players, desc="Scraping players")):


Scraping players: 0it [00:00, ?it/s]

        School     Conf Class Pos   G  Att   Yds  Avg  TD Rec  Yds   Avg TD  \
0     Virginia      ACC        RB  11   36   205  5.7   4   4   26   6.5  0   
1     Virginia      ACC        RB  11  201   692  3.4   4  17  127   7.5  1   
2     Virginia      ACC        RB  11  238  1303  5.5  13  28  179   6.4  2   
3     Virginia      ACC        RB  11  334  1798  5.4  16  22  239  10.9  1   
0    Wisconsin  Big Ten        RB  13  325  2109  6.5  21  14  133   9.5  0   
..         ...      ...   ...  ..  ..  ...   ...  ...  ..  ..  ...   ... ..   
3   Ohio State  Big Ten    JR  RB  13  172   954  5.5   5  21  112   5.3  1   
0   Washington   Pac-12    FR  RB  13  227  1302  5.7  14   6   19   3.2  0   
1   Washington   Pac-12    SO  RB  14  237  1373  5.8  10  19  137   7.2  1   
2   Washington   Pac-12    JR  RB  13  222  1380  6.2  21  19  232  12.2  3   
3   Washington   Pac-12    SR  RB  12  259  1268  4.9  12  21   77   3.7  1   

   Plays   Yds  Avg  TD        Player  
0     40   

In [ ]:
college_rb_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb/rb_college_output.csv') 


# Fantasy Football Points Scraping



In [ ]:
# ## For each player obtain their college stats
# # Create a list to hold the college stats dataframes
# fantasy_rb_pts_df = []


# YEARS = list(range(2001,2022))

# # Loop through each player in the draft results dataframe
# for year in YEARS:
#     try:
#         # Create a URL for the player's college stats page
#         url = f"https://fantasydata.com/nfl/fantasy-football-leaders?position=3&season={year}&seasontype=1&scope=1&subscope=1&startweek=1&endweek=1&aggregatescope=1&range=1"
#         print(url)
#         # Request the HTML content from the URL
#         response = requests.get(url)

#         # Parse the HTML content using BeautifulSoup
#         soup = BeautifulSoup(response.content, "html.parser")

#         # Find the table element containing the player's college stats. Rb's table is called rushing
#         table = soup.find("table", {"id": "stats_grid"})
#         print(table)

#         # Extract the table headers
#         headers = [th.get_text() for th in table.find("thead").find_all("th")]
#         # headers = headers[6:]

#         # Create a list to hold the college stats data
#         yearly_stats = []

#         # Extract the college stats row by row
#         for tr in table.find("tbody").find_all("tr"):
#             row = [td.get_text() for td in tr.find_all("td")]
#             yearly_stats.append(row)

#         # Convert the college stats to a pandas DataFrame
#         fantasy_year_df = pd.DataFrame(yearly_stats, columns=headers)

#         # Add a "Player" column to the college stats DataFrame
#         fantasy_year_df["Year"] = year

#         # Add the college stats DataFrame to the list of college stats dataframes
#         fantasy_rb_pts_df.append(fantasy_year_df)

#         # Create first pandas dataframe and then add subsequent dataframes
#         if len(college_rb_dfs)==1:
#             fantasy_rb_pts_df = fantasy_year_df
#         else:
#             fantasy_rb_pts_df = pd.concat([fantasy_rb_pts_df,fantasy_year_df])      

#     except:
#         # Notification of years that couldn't be read in
#         print("error year:")
#         print(year)

## Consolidating Dataset

The datasets will be merged into a single wr and A single rb dataset. The set of players will stick to the limits of the nfl drafted rbs and wrs. The combine data will me merged left to this draft dataset. 

The fantasy dataset will be prepped first, since we will not need to predict the specifics of how each player will reach the projected fpts total, but rather just this individual total will be predicted.

It is important how we choose to predict the fpts. We could predict total fpts overall, however, role players that are able to stay in the league a long time but fail to produce a high peak of their careers are rewarded in this model more that players have a high "ceiling". This model seeks to predict a player's peak since high performing players with low years in the year, but are highly succesful in those years are more valuable than the low consistent players.

The predicted column will be a combination of many successful years and how successful the player is in those respective years. We will choose how many top 24 seasons they produce in their careers. To achieve this number, a player must be in the top 24 of their cumaltive sum of fantasy points scored with respect to their position. Since their are 32 teams, this statistic typically implies if they are best wide receiver or running back not only on their team, but also are almost a top 2/3 starting running back or wide receiver in the league. 

In [ ]:
# Summarizing fantasy points scored data into number of top 24 running back or top 24 wide receiver season. 
# Define a list to store the filtered data for each year
rb_fantasy = []
wr_fantasy = []

# Loop over the list of years
for year in range(2001, 2022):
  # try:
    # Define the file name for the current year
    file_name = f"/content/drive/MyDrive/Colab Notebooks/Data-Collection/Fantasy/{year}.csv"
    
    # Load the csv file into a pandas dataframe
    df = pd.read_csv(file_name)
    
    # Filter the data where the "pos" column is either "RB" or "WR"
    rb_df = df[df['Pos'] == 'RB']
    wr_df = df[df['Pos'] == 'WR']
    
    # Sort the data by "FantasyPoints" column in descending order
    rb_df = rb_df.sort_values('FantasyPoints', ascending=False).reset_index(drop=True)
    wr_df = wr_df.sort_values('FantasyPoints', ascending=False).reset_index(drop=True)
    
    # Select only the top 24 players and keep only the "FantasyPoints" and "Player" columns
    rb_df = rb_df[['FantasyPoints', 'Player']].head(24)
    wr_df = wr_df[['FantasyPoints', 'Player']].head(24)
    
    # Append the filtered data to the list for each position
    rb_fantasy.append(rb_df)
    wr_fantasy.append(wr_df)
  # except:
    # print("Error Year: " + str(year))


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Data-Collection/Fantasy/'

# Concatenate the dataframes vertically
rb_fantasy = pd.concat(rb_fantasy)
wr_fantasy = pd.concat(wr_fantasy)

# Count the number of top 24 seasons for each player
rb_counts = rb_fantasy['Player'].value_counts()
wr_counts = wr_fantasy['Player'].value_counts()

# Create new dataframes with two columns: "Player" and "Top_24_Seasons"
rb_summary = pd.DataFrame({'Player': rb_counts.index, 'Top_24_Seasons': rb_counts.values})
wr_summary = pd.DataFrame({'Player': wr_counts.index, 'Top_24_Seasons': wr_counts.values})

# Write the summary dataframes to csv files
rb_summary.to_csv(file_path + 'RB_summary.csv', index=False)
wr_summary.to_csv(file_path + 'WR_summary.csv', index=False)

# Write the combined dataframes to csv files
rb_fantasy.to_csv(file_path + 'RB_top_24_players.csv', index=False)
wr_fantasy.to_csv(file_path + 'WR_top_24_players.csv', index=False)

## Merging Datasets

In [ ]:
# Merge draft with fantasy points scored.

# Define the file path to the directory containing the csv files
file_path = '/content/drive/MyDrive/Colab Notebooks/Data-Collection/Fantasy/'

# Load the rb and wr summary dataframes
rb_summary = pd.read_csv(file_path + 'RB_summary.csv')
wr_summary = pd.read_csv(file_path + 'WR_summary.csv')

# Load the rb and wr draft dataframes
rb_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb/rb_draft_output.csv')
wr_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_draft_output.csv')
rb_combine = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb/rb_combine_output.csv')
wr_combine = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_combine_output.csv')

# Merge original with fantasy
rb_merged = pd.merge(rb_df, rb_summary, on='Player', how='left')
wr_merged = pd.merge(wr_df, wr_summary, on='Player', how='left')

# Fill in NaN values with 0
rb_merged['Top_24_Seasons'] = rb_merged['Top_24_Seasons'].fillna(0)
wr_merged['Top_24_Seasons'] = wr_merged['Top_24_Seasons'].fillna(0)


# Merge with combine
rb_combine_df = pd.merge(rb_merged, rb_summary, on='Player', how='left')
wr_combine_df = pd.merge(wr_merged, wr_summary, on='Player', how='left')


# Add in Combine Dataset using left merge on orginal draft names
rb_merged_combine = pd.merge(rb_merged, rb_combine, on='Player', how='left')
wr_merged_combine = pd.merge(wr_merged, wr_combine, on='Player', how='left')

rb_college_total = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb/rb_college_total.csv')
rb_college_total.rename(columns={rb_college_total.columns[0]: "CollegeYear"}, inplace=True)

wr_college_total = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_college_total.csv')
wr_college_total.rename(columns={wr_college_total.columns[0]: "CollegeYear"}, inplace=True)



In [ ]:
# Display the dataset feature variable types and null values
tab_info=pd.DataFrame(rb_college_total.dtypes).T.rename(index={0:'Variable Type'})
tab_info=tab_info.append(pd.DataFrame(rb_college_total.isnull().sum()).T.rename(index={0:'Null Values'}))
tab_info = tab_info.append(pd.DataFrame(rb_college_total.isnull().sum()/rb_college_total.shape[0]*100).T.rename(index={0:'Null Value (%)'}))
tab_info

,CollegeYear,School,Conf,Class,Pos,G,Att,Yds,Avg,TD,Rec,Yds.1,Avg.1,TD.1,Plays,Yds.2,Avg.2,TD.2,Player
Variable Type,int64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,object
Null Values,0,0,0,290,0,4,24,24,30,24,128,128,156,128,0,0,22,0,0
Null Value (%),0.0,0.0,0.0,13.086643,0.0,0.180505,1.083032,1.083032,1.353791,1.083032,5.776173,5.776173,7.039711,5.776173,0.0,0.0,0.99278,0.0,0.0


In [ ]:
# Display the dataset feature variable types and null values
tab_info=pd.DataFrame(wr_college_total.dtypes).T.rename(index={0:'Variable Type'})
tab_info=tab_info.append(pd.DataFrame(wr_college_total.isnull().sum()).T.rename(index={0:'Null Values'}))
tab_info = tab_info.append(pd.DataFrame(wr_college_total.isnull().sum()/wr_college_total.shape[0]*100).T.rename(index={0:'Null Value (%)'}))
tab_info

,CollegeYear,School,Conf,Class,Pos,G,Rec,Yds,Avg,TD,Att,Yds.1,Avg.1,TD.1,Plays,Yds.2,Avg.2,TD.2,Player
Variable Type,int64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,object
Null Values,0,0,0,245,0,11,25,25,33,25,552,552,723,552,0,0,27,0,0
Null Value (%),0.0,0.0,0.0,14.741276,0.0,0.661853,1.504212,1.504212,1.98556,1.504212,33.212996,33.212996,43.501805,33.212996,0.0,0.0,1.624549,0.0,0.0


In [ ]:
# Group by 'Player', exclude non-numeric columns, and apply mean() to the remaining columns
numeric_cols = ['G', 'Att', 'Yds', 'Avg', 'TD', 'Rec', 'Yds.1', 'Avg.1', 'TD.1', 'Plays', 'Yds.2', 'Avg.2', 'TD.2']
agg_funcs = {col: 'mean' for col in numeric_cols}

rb_college_average = rb_college_total.groupby('Player', as_index=False).agg(agg_funcs)
wr_college_average = wr_college_total.groupby('Player', as_index=False).agg(agg_funcs)

# Get the last values of 'School', 'Conf', 'Class', and 'Pos' for each group
last_values = rb_college_total.groupby('Player').last().reset_index()[['CollegeYear', 'School', 'Conf', 'Class', 'Pos']]
last_values_wr = wr_college_total.groupby('Player').last().reset_index()[['CollegeYear', 'School', 'Conf', 'Class', 'Pos']]

# Add the last values to the new dataframe
rb_college_average[['CollegeYear', 'School', 'Conf', 'Class', 'Pos']] = last_values
wr_college_average[['CollegeYear', 'School', 'Conf', 'Class', 'Pos']] = last_values_wr

rb_college_average['CollegeYear'] = rb_college_average['CollegeYear']  + 1
wr_college_average['CollegeYear'] = wr_college_average['CollegeYear']  + 1

rb_college_average.head()

,Player,G,Att,Yds,Avg,TD,Rec,Yds.1,Avg.1,TD.1,Plays,Yds.2,Avg.2,TD.2,CollegeYear,School,Conf,Class,Pos
0,Adimchinobe Echemandu,13.000000,238.000000,1195.000000,5.000000,13.000000,22.000000,185.000000,8.400000,0.000000,260.000000,1380.000000,5.300000,13.00,1,California,Pac-10,SR,RB
1,Adrian Peterson,10.333333,249.333333,1347.000000,5.366667,13.666667,8.000000,66.000000,7.200000,0.333333,257.333333,1413.000000,5.466667,14.00,3,Oklahoma,Big 12,JR,RB
2,Ahmaad Galloway,9.500000,94.000000,457.500000,4.750000,4.250000,2.666667,24.333333,7.900000,0.000000,96.000000,475.750000,4.800000,4.25,4,Alabama,SEC,SR,RB
3,Ahmad Bradshaw,11.000000,186.000000,994.000000,5.233333,10.333333,29.000000,232.333333,9.266667,1.666667,215.000000,1226.333333,5.766667,12.00,3,Marshall,CUSA,JR,RB
4,Alex Collins,12.666667,221.666667,1234.333333,5.533333,12.000000,9.000000,55.666667,5.333333,0.000000,230.666667,1290.000000,5.566667,12.00,3,Arkansas,SEC,JR,RB


In [ ]:
# Merge with College
rb_merged_college = pd.merge(rb_merged_combine, rb_college_average, on='Player', how='left')
rb_merged_college = rb_merged_college.dropna(subset=['Avg'])
rb_merged_college.drop_duplicates()
rb_merged_college['CollegeYear'] = rb_merged_college['CollegeYear'].replace(0, 1)

wr_merged_college = pd.merge(wr_merged_combine, wr_college_average, on='Player', how='left')
wr_merged_college.drop_duplicates()
wr_merged_college['CollegeYear'] = wr_merged_college['CollegeYear'].replace(0, 1)
wr_merged_college = wr_merged_college.dropna(subset=['Avg'])

In [ ]:
rb_merged_college.head()

,Unnamed: 0_x,Pick,Tm,Player,Pos_x,Age,To,AP1,PB,St,...,TD.1_y,Plays,Yds.2_y,Avg.2,TD.2_y,CollegeYear,School_y,Conf,Class,Pos
1,6,7,ARI,Thomas Jones,RB,22.0,2011.0,0,1,9,...,1.000000,220.00,1142.25,5.225000,10.25,4.0,Virginia,ACC,None,RB
2,10,11,NYG,Ron Dayne,RB,22.0,2007.0,0,0,1,...,0.000000,312.75,1857.25,5.900000,17.75,4.0,Wisconsin,Big Ten,None,RB
3,18,19,SEA,Shaun Alexander,RB,23.0,2008.0,1,3,7,...,2.000000,197.25,1090.75,5.775000,12.25,4.0,Alabama,SEC,None,RB
4,30,31,STL,Trung Canidate,RB,23.0,2003.0,0,0,1,...,0.333333,150.00,1023.50,6.833333,6.50,4.0,Arizona,Pac-10,None,RB
5,64,63,CLE,Travis Prentice,RB,23.0,2001.0,0,0,1,...,1.250000,298.00,1529.50,5.100000,19.50,4.0,Miami (OH),MAC,None,RB


In [ ]:
rb_merged_college = rb_merged_college.drop(['Unnamed: 0_x','Tm','Pos_x','To','AP1','PB','St','Cmp','Att_x','Yds_x','TD_x','Int', 'Solo',
       'Int.1','Sk','College/Univ','Unnamed: 28', 'Pos_y', 'School_x', 'College', 'Drafted (tm/rnd/yr)', 'School_y', 'Conf', 
       'Class', 'Pos'], axis=1)

wr_merged_college = wr_merged_college.drop(['Unnamed: 0_x','Tm','Pos_x','To','AP1','PB','St','Cmp','Att_x','Yds_x','TD_x','Int', 'Solo',
       'Int.1','Sk','College/Univ','Unnamed: 28', 'Pos_y', 'School_x', 'College', 'Drafted (tm/rnd/yr)', 'School_y', 'Conf', 
       'Class', 'Pos'], axis=1)

In [ ]:
# check if 'Jamal Lewis' has a NaN value for 'TD.1_y'
has_nan = rb_merged_college.loc[rb_merged_college['Player'] == 'Jamal Lewis', 'TD.1_y'].isna().values[0]
print(has_nan)

IndexError: ignored

In [ ]:
print(rb_merged_college.columns)

Index(['Pick', 'Player', 'Age', 'wAV', 'DrAV', 'G_x', 'Att.1', 'Yds.1_x',
       'TD.1_x', 'Rec_x', 'Yds.2_x', 'TD.2_x', 'Top_24_Seasons',
       'Unnamed: 0_y', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump',
       '3Cone', 'Shuttle', 'G_y', 'Att_y', 'Yds_y', 'Avg', 'TD_y', 'Rec_y',
       'Yds.1_y', 'Avg.1', 'TD.1_y', 'Plays', 'Yds.2_y', 'Avg.2', 'TD.2_y',
       'CollegeYear'],
      dtype='object')


In [ ]:
rb_merged_college['G_y'] = np.where(rb_merged_college['G_y'].isnull() | rb_merged_college['G_y'].isna(), rb_merged_college['G_x']/rb_merged_college['CollegeYear'], rb_merged_college['G_y'])
rb_merged_college['Att_y'] = np.where(rb_merged_college['Att_y'].isnull() | rb_merged_college['Att_y'].isna(), rb_merged_college['Att.1']/rb_merged_college['CollegeYear'], rb_merged_college['Att_y'])
rb_merged_college['Yds_y'] = np.where(rb_merged_college['Yds_y'].isnull() | rb_merged_college['Yds_y'].isna(), rb_merged_college['Yds.1_x']/rb_merged_college['CollegeYear'], rb_merged_college['Yds_y'])
rb_merged_college['TD_y'] = np.where(rb_merged_college['TD_y'].isnull() | rb_merged_college['TD_y'].isna(), rb_merged_college['TD.1_x']/rb_merged_college['CollegeYear'], rb_merged_college['TD_y'])
rb_merged_college['Rec_y'] = np.where(rb_merged_college['Rec_y'].isnull() | rb_merged_college['Rec_y'].isna(), rb_merged_college['Rec_x']/rb_merged_college['CollegeYear'], rb_merged_college['Rec_y'])
rb_merged_college['Yds.1_y'] = np.where(rb_merged_college['Yds.1_y'].isnull() | rb_merged_college['Yds.1_y'].isna(), rb_merged_college['Yds.2_x']/rb_merged_college['CollegeYear'], rb_merged_college['Yds.1_y'])
rb_merged_college['TD.1_y'] = np.where(rb_merged_college['TD.1_y'].isnull() | rb_merged_college['TD.1_y'].isna(), rb_merged_college['TD.2_x']/rb_merged_college['CollegeYear'], rb_merged_college['TD.1_y'])

In [ ]:
# check if 'Jamal Lewis' has a NaN value for 'TD.1_y'
has_nan = rb_merged_college.loc[rb_merged_college['Player'] == 'Jamal Lewis', 'TD.1_y'].isna().values[0]
print(has_nan)

# find the row index of the player 'Jamal Lewis'
jamal_row_index = rb_merged_college.index[rb_merged_college['Player'] == 'Jamal Lewis'][0]

# get the value of the 'Ht' column for Jamal Lewis
jamal_height = rb_merged_college.loc[jamal_row_index, 'Ht']

# print the type and value of the 'Ht' column for Jamal Lewis
print(type(jamal_height))
print(jamal_height)

IndexError: ignored

In [ ]:
# define a function to convert height to inches
def convert_to_inches(height):
    if isinstance(height, float):
        return height
    feet, inches = height.split('-')
    return (int(feet) * 12) + int(inches)

# apply the function to the height column and convert to float type
rb_merged_college['Ht'] = rb_merged_college['Ht'].apply(convert_to_inches).astype(float)
wr_merged_college['Ht'] = wr_merged_college['Ht'].apply(convert_to_inches).astype(float)



## Interpolate values with KNNInputer and Linear Inputation


In [ ]:
from sklearn.impute import KNNImputer

rb_merged_college['Age']=rb_merged_college['Age'].interpolate(method='linear',limit_direction='both',axis=0)
rb_merged_college['wAV']=rb_merged_college['wAV'].interpolate(method='linear',limit_direction='both',axis=0)
rb_merged_college['DrAV']=rb_merged_college['DrAV'].interpolate(method='linear',limit_direction='both',axis=0)
rb_merged_college['Wt']=rb_merged_college['Wt'].interpolate(method='linear',limit_direction='both',axis=0)
rb_merged_college['Ht']=rb_merged_college['Ht'].interpolate(method='linear',limit_direction='both',axis=0)


# select the columns to interpolate
college_columns_to_interpolate = ['G_y', 'Att_y', 'Yds_y', 'Avg', 'TD_y', 'Rec_y', 'Yds.1_y', 'Avg.1', 'TD.1_y', 'Plays', 'CollegeYear']
combine_columns_to_interpolate = ['40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle']


# create a KNN imputer object with 3 nearest neighbors
imputer = KNNImputer(n_neighbors=3)

# interpolate the columns
rb_merged_college[college_columns_to_interpolate] = imputer.fit_transform(rb_merged_college[college_columns_to_interpolate])
rb_merged_college[combine_columns_to_interpolate] = imputer.fit_transform(rb_merged_college[combine_columns_to_interpolate])

rb_merged_college = rb_merged_college.drop(['Unnamed: 0_y', 'TD.2_y', 'Avg.2', 'Yds.2_y', 'G_x', 'Att.1', 'Yds.1_x',
       'TD.1_x', 'Rec_x', 'Yds.2_x', 'TD.2_x'], axis=1)


# Now repeat for wr
wr_merged_college['Age']=wr_merged_college['Age'].interpolate(method='linear',limit_direction='both',axis=0)
wr_merged_college['wAV']=wr_merged_college['wAV'].interpolate(method='linear',limit_direction='both',axis=0)
wr_merged_college['DrAV']=wr_merged_college['DrAV'].interpolate(method='linear',limit_direction='both',axis=0)
wr_merged_college['Wt']=wr_merged_college['Wt'].interpolate(method='linear',limit_direction='both',axis=0)
wr_merged_college['Ht']=wr_merged_college['Ht'].interpolate(method='linear',limit_direction='both',axis=0)


# select the columns to interpolate
college_columns_to_interpolate = ['G_y', 'Att_y', 'Yds_y', 'Avg', 'TD_y', 'Rec_y', 'Yds.1_y', 'Avg.1', 'TD.1_y', 'Plays', 'CollegeYear']
combine_columns_to_interpolate = ['40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle']


# create a KNN imputer object with 3 nearest neighbors
imputer_wr = KNNImputer(n_neighbors=3)

# interpolate the columns
wr_merged_college[college_columns_to_interpolate] = imputer_wr.fit_transform(wr_merged_college[college_columns_to_interpolate])
wr_merged_college[combine_columns_to_interpolate] = imputer_wr.fit_transform(wr_merged_college[combine_columns_to_interpolate])

wr_merged_college = wr_merged_college.drop(['Unnamed: 0_y', 'TD.2_y', 'Avg.2', 'Yds.2_y', 'G_x', 'Att.1', 'Yds.1_x',
       'TD.1_x', 'Rec_x', 'Yds.2_x', 'TD.2_x'], axis=1)


In [ ]:
# Rename columns to more interpretable title
column_mapping = {
    'G_y': 'Games',
    'Att_y': 'Rush_Att',
    'Yds_y': 'Rush_Yds',
    'Avg': 'RushYds_Avg',
    'TD_y': 'Rush_TD',
    'Rec_y': 'Rec',
    'Yds.1_y': 'Rec_Yds',
    'Avg.1': 'RecYds_Avg',
    'TD.1_y': 'Rec_TD'
}


rb_merged_college = rb_merged_college.rename(columns=column_mapping)
wr_merged_college = wr_merged_college.rename(columns=column_mapping)

In [ ]:
# Write the merged dataframes to csv files
rb_merged_college.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/rb/rb_college_summary.csv', index=False)
wr_merged_college.to_csv('/content/drive/MyDrive/Colab Notebooks/Data-Collection/wr/wr_college_summary.csv', index=False)

In [ ]:
rb_merged_college.head()

,Pick,Player,Age,wAV,DrAV,Top_24_Seasons,Ht,Wt,40yd,Vertical,...,Rush_Att,Rush_Yds,RushYds_Avg,Rush_TD,Rec,Rec_Yds,RecYds_Avg,Rec_TD,Plays,CollegeYear
1,7,Thomas Jones,22.0,62.0,7.0,6.0,70.0,216.0,4.45,36.333333,...,202.25,999.500000,5.000000,9.250000,17.75,142.75,7.825000,1.000000,220.00,4.0
2,11,Ron Dayne,22.0,23.0,10.0,0.0,71.0,259.0,4.65,35.000000,...,305.00,1781.250000,5.800000,17.750000,7.75,76.00,9.425000,0.000000,312.75,4.0
3,19,Shaun Alexander,23.0,68.0,68.0,5.0,72.0,218.0,4.58,33.833333,...,181.75,891.250000,5.350000,10.250000,15.50,199.50,11.150000,2.000000,197.25,4.0
4,31,Trung Canidate,23.0,11.0,6.0,0.0,71.0,193.0,4.41,37.833333,...,186.00,1208.666667,6.466667,8.333333,14.00,156.00,12.533333,0.333333,150.00,4.0
5,63,Travis Prentice,23.0,4.0,4.0,0.0,72.0,225.0,4.50,37.000000,...,284.50,1399.000000,4.925000,18.250000,13.50,130.50,9.225000,1.250000,298.00,4.0


In [ ]:
wr_merged_college.head()

,Pick,Player,Age,wAV,DrAV,Top_24_Seasons,Ht,Wt,40yd,Vertical,...,Rush_Att,Rush_Yds,RushYds_Avg,Rush_TD,Rec,Rec_Yds,RecYds_Avg,Rec_TD,Plays,CollegeYear
0,4,Peter Warrick,23.0,27.0,25.0,1.0,71.0,194.0,4.58,35.000000,...,8.200000,879.250000,17.825,8.0,51.75,37.600000,2.540000,0.800000,49.600000,5.0
1,8,Plaxico Burress,23.0,70.0,34.0,4.0,77.0,231.0,4.59,33.000000,...,2.333333,1077.500000,16.450,10.0,65.50,13.833333,6.350000,0.166667,65.500000,2.0
2,10,Travis Taylor,22.0,30.0,23.0,0.0,73.0,199.0,4.43,37.000000,...,4.000000,383.333333,14.300,5.0,24.00,25.000000,6.300000,0.000000,25.333333,3.0
5,32,Dennis Northcutt,22.0,41.0,28.0,0.0,70.0,175.0,4.43,37.500000,...,8.500000,796.500000,13.350,6.0,54.25,95.500000,10.275000,0.500000,62.750000,4.0
6,36,Todd Pinkston,23.0,25.0,25.0,0.0,74.0,169.0,4.45,36.833333,...,4.250000,591.500000,15.675,5.5,37.25,46.083333,5.991667,0.333333,37.250000,4.0


In [ ]:
tab_info2=pd.DataFrame(rb_merged_college.dtypes).T.rename(index={0:'Variable Type'})
tab_info2=tab_info2.append(pd.DataFrame(rb_merged_college.isnull().sum()).T.rename(index={0:'Null Values'}))
tab_info2 = tab_info2.append(pd.DataFrame(rb_merged_college.isnull().sum()/rb_merged_college.shape[0]*100).T.rename(index={0:'Null Value (%)'}))
tab_info2

,Pick,Player,Age,wAV,DrAV,Top_24_Seasons,Ht,Wt,40yd,Vertical,...,Rush_Att,Rush_Yds,RushYds_Avg,Rush_TD,Rec,Rec_Yds,RecYds_Avg,Rec_TD,Plays,CollegeYear
Variable Type,int64,object,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Null Values,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Null Value (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tab_info2=pd.DataFrame(wr_merged_college.dtypes).T.rename(index={0:'Variable Type'})
tab_info2=tab_info2.append(pd.DataFrame(wr_merged_college.isnull().sum()).T.rename(index={0:'Null Values'}))
tab_info2 = tab_info2.append(pd.DataFrame(wr_merged_college.isnull().sum()/wr_merged_college.shape[0]*100).T.rename(index={0:'Null Value (%)'}))
tab_info2

,Pick,Player,Age,wAV,DrAV,Top_24_Seasons,Ht,Wt,40yd,Vertical,...,Rush_Att,Rush_Yds,RushYds_Avg,Rush_TD,Rec,Rec_Yds,RecYds_Avg,Rec_TD,Plays,CollegeYear
Variable Type,int64,object,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Null Values,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Null Value (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
